# Dynamic Programming Algorithms - Experiments - Chapter 5 and 6.2

In [ ]:
import os
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt

from typing import List, Optional
from shapely.geometry import Polygon, MultiPolygon, Point

from experimental.utils.dynamic_system import DynamicSystem
from experimental.utils.markov_decision_process import MarkovDecisionProcess

## Build equally-spaced state space partitions

In [ ]:
# load baseline partitions from disk, if they already exist
if os.path.isfile("results/baseline_partitions.pkl"):
    with open("results/baseline_partitions.pkl", "rb") as file:
        baseline_partitions = pickle.load(file)
else:
    baseline_partitions = {}

baseline_partitions.keys()

In [ ]:
n = 3
m_id = 1
equally_spaced_offset_x = np.array([m_id/3, 0])
equally_spaced_offset_y = np.array([0, m_id/3])

In [ ]:
u1 = np.array([0,0])
u2 = np.array([0,1])
u3 = np.array([1,1])
u4 = np.array([1,0])

In [ ]:
x1 = u1 + equally_spaced_offset_x
x2 = u1 + 2*equally_spaced_offset_x
x3 = u2 + equally_spaced_offset_x
x4 = u2 + 2*equally_spaced_offset_x

In [ ]:
X1_vertices = [u1, u2, x3, x1]
X1 = MultiPolygon([Polygon(X1_vertices)])

X2_vertices = [x3, x4, x2, x1]
X2 = MultiPolygon([Polygon(X2_vertices)])

X3_vertices = [x4, u3, u4, x2]
X3 = MultiPolygon([Polygon(X3_vertices)])

In [ ]:
MultiPolygon([Polygon([u1, u2, x3, x1]), Polygon([x3, x4, x2, x1]), Polygon([x4, u3, u4, x2])])

In [ ]:
horizontal_partition = [X1, X2, X3]
horizontal_partition_vertices = [X1_vertices, X2_vertices, X3_vertices]

baseline_partitions["horizontal"] = {}
baseline_partitions["horizontal"]["partition"] = horizontal_partition
baseline_partitions["horizontal"]["partition_vertices"] = horizontal_partition_vertices

In [ ]:
y1 = u1 + equally_spaced_offset_y
y2 = u1 + 2*equally_spaced_offset_y
y3 = u4 + equally_spaced_offset_y
y4 = u4 + 2*equally_spaced_offset_y

In [ ]:
Y1_vertices = [u1, y1, y3, u4]
Y1 = MultiPolygon([Polygon(Y1_vertices)])

Y2_vertices = [y1, y2, y4, y3]
Y2 = MultiPolygon([Polygon(Y2_vertices)])

Y3_vertices = [y2, u2, u3, y4]
Y3 = MultiPolygon([Polygon(Y3_vertices)])

In [ ]:
MultiPolygon([Polygon([u1, y1, y3, u4]), Polygon([y1, y2, y4, y3]), Polygon([y2, u2, u3, y4])])

In [ ]:
vertical_partition = [Y1, Y2, Y3]
vertical_partition_vertices = [Y1_vertices, Y2_vertices, Y3_vertices]

baseline_partitions["vertical"] = {}
baseline_partitions["vertical"]["partition"] = vertical_partition
baseline_partitions["vertical"]["partition_vertices"] = vertical_partition_vertices

In [ ]:
x_halved = np.array([m_id/2, 0])

In [ ]:
A1_vertices = [u2, u3, y4, y2]
A1 = MultiPolygon([Polygon(A1_vertices)])

A2_vertices = [u1, y2, y2+x_halved, u1+x_halved]
A2 = MultiPolygon([Polygon(A2_vertices)])

A3_vertices = [u1+x_halved, y2+x_halved, y4, u4]
A3 = MultiPolygon([Polygon(A3_vertices)])

In [ ]:
MultiPolygon([Polygon([u2, u3, y4, y2]), Polygon([u1, y2, y2+x_halved, u1+x_halved]), Polygon([u1+x_halved, y2+x_halved, y4, u4])])

In [ ]:
complex_partition_1 = [A1, A2, A3]
complex_partition_vertices_1 = [A1_vertices, A2_vertices, A3_vertices]

baseline_partitions["complex_1"] = {}
baseline_partitions["complex_1"]["partition"] = complex_partition_1
baseline_partitions["complex_1"]["partition_vertices"] = complex_partition_vertices_1

In [ ]:
y_halved = np.array([0, m_id/2])

In [ ]:
B1_vertices = [u1, u1+y_halved, x2+y_halved, x2]
B1 = MultiPolygon([Polygon(B1_vertices)])

B2_vertices = [u1+y_halved, u2, x4, x2+y_halved]
B2 = MultiPolygon([Polygon(B2_vertices)])

B3_vertices = [x4, u3, u4, x2]
B3 = MultiPolygon([Polygon(B3_vertices)])

In [ ]:
MultiPolygon([Polygon([u1, u1+y_halved, x2+y_halved, x2]), Polygon([u1+y_halved, u2, x4, x2+y_halved]), Polygon([x4, u3, u4, x2])])

In [ ]:
complex_partition_2 = [B1, B2, B3]
complex_partition_vertices_2 = [B1_vertices, B2_vertices, B3_vertices]

baseline_partitions["complex_2"] = {}
baseline_partitions["complex_2"]["partition"] = complex_partition_2
baseline_partitions["complex_2"]["partition_vertices"] = complex_partition_vertices_2

In [ ]:
baseline_partitions.keys()

In [ ]:
with open(f"results/baseline_partitions.pkl", "wb") as file:
    pickle.dump(baseline_partitions, file, protocol=pickle.HIGHEST_PROTOCOL)

## Test Policy Evaluation Algorithm with standard example dynamic system

In [ ]:
A = np.array([[1,1], [1,0]])

In [ ]:
def phi(x: np.array) -> np.array:
    x_new = np.dot(A, x) % 1
    return x_new

In [ ]:
def q(x: np.array) -> np.array:
    return x % 1

In [ ]:
# get eigenvalues and right eigenvectors, i.e. transposed eigenvectors
eig_vals, eig_vects = np.linalg.eig(A)
# retrieve eigenvectors from right eigenvectors
eig_vects = np.transpose(eig_vects)

In [ ]:
print(f"Eigenvalues: {eig_vals}")

In [ ]:
print(f"Eigenvectors: {eig_vects}")

In [ ]:
v1 = eig_vects[0]
v2 = eig_vects[1]

In [ ]:
def perp(a) :
    b = np.empty_like(a)
    b[0] = -a[1]
    b[1] = a[0]
    return b

# line segment a given by endpoints a1, a2
# line segment b given by endpoints b1, b2
def seg_intersect(a, b) :
    a1, a2 = a
    b1, b2 = b

    da = a2 - a1
    db = b2 - b1
    dp = a1 - b1
    dap = perp(da)
    denom = np.dot(dap, db)
    num = np.dot(dap, dp)

    return (num / denom.astype(float))*db + b1

In [ ]:
u1 = np.array([0,0])
u2 = np.array([0,1])
u3 = np.array([1,1])
u4 = np.array([1,0])

In [ ]:
lu1 = np.array([[0,0], [0,1]])
lu2 = np.array([[0,1], [1,1]])
lu3 = np.array([[1,1], [1,0]])
lu4 = np.array([[1,0], [0,0]])

In [ ]:
p1 = u1 + (1/v1[0]) * v1
p2 = u2 - (1/v2[1]) * v2
p3 = u3 - (1/v1[0]) * v1
p4 = u4 + (1/v2[1]) * v2

In [ ]:
l_00 = np.array([u1, p1])
l_01 = np.array([u2, p2])
l_10 = np.array([u4, p4])
l_11 = np.array([u3, p3])

In [ ]:
P1a = seg_intersect(l_00, l_01)
P1b = seg_intersect(l_10, l_00)

P3a = seg_intersect(l_10, l_11)

symm_helper_x = seg_intersect(lu2, l_10)
symm_helper_y = symm_helper_x - np.array([0,1])
symm_helper_l11 = np.array([symm_helper_x, symm_helper_y])

p1_symm = symm_helper_y + v2
l_10_symm_extension = np.array([symm_helper_y, p1_symm])
P3b = seg_intersect(l_10_symm_extension, l_00)
l_10_symm_extension = np.array([symm_helper_y, P3b])

In [ ]:
l_00 = np.array([u1, P1b])
l_01 = np.array([u2, P1a])
l_10 = np.array([u4, p4])
l_11 = np.array([u3, P3a])

In [ ]:
# plot unit square
plt.plot(lu1[:, 0], lu1[:, 1], "r-")
plt.plot(lu2[:, 0], lu2[:, 1], "r-")
plt.plot(lu3[:, 0], lu3[:, 1], "r-")
plt.plot(lu4[:, 0], lu4[:, 1], "r-")

# step 1: plot l_10 in contracting direction
plt.plot(l_10[:, 0], l_10[:, 1], "b-")

# step 2: plot l_00 and l_11 in expanding directions
plt.plot(l_00[:, 0], l_00[:, 1], "b-")
plt.plot(l_11[:, 0], l_11[:, 1], "b-")

# step 3: plot l_01 in contracting direction 
plt.plot(l_01[:, 0], l_01[:, 1], "b-")

# step 4: plot symmetric extension of l_01 line
plt.plot(symm_helper_l11[:, 0], symm_helper_l11[:, 1], "m--")
plt.plot(l_10_symm_extension[:, 0], l_10_symm_extension[:, 1], "b-")

# plot intersection points
plt.plot(P1a[0], P1a[1], "bo")
plt.plot(P1b[0], P1b[1], "ro")
plt.plot(P3a[0], P3a[1], "go")
plt.plot(P3b[0], P3b[1], "yo")

plt.show()

In [ ]:
P1A = Polygon([P1a, u1, u2])
P1B = Polygon([P3a, u3, u4])
P1 = MultiPolygon([P1A, P1B])

In [ ]:
P2A = Polygon([P3b, u1, symm_helper_l11[1]])
P2B = Polygon([symm_helper_l11[0], u2, P1a, P1b])
P2 = MultiPolygon([P2A, P2B])

In [ ]:
P3A = Polygon([P1b, u4, symm_helper_l11[1], P3b])
P3B = Polygon([symm_helper_l11[0], u3, P3a])
P3 = MultiPolygon([P3A, P3B])

In [ ]:
P1_vertices = [P1a, u1, u2, P3a]
P2_vertices = [P3b, u1, P1a, P1b]
P3_vertices = [P1b, u4, P3b, P3a]
partition_vertices = [P1_vertices, P2_vertices, P3_vertices]

In [ ]:
partition = [P1, P2, P3]

In [ ]:
MultiPolygon([P1A, P1B, P2A, P2B, P3A, P3B])

In [ ]:
P1A_phi = Polygon([P1b, u4, u1])
P1B_phi = Polygon([u2, u3, phi(P3a)])
P1_phi = MultiPolygon([P1A_phi, P1B_phi])

In [ ]:
symm_helper_x = seg_intersect(lu3, l_00)
symm_helper_y = q(symm_helper_x)
symm_helper_l00 = np.array([symm_helper_x, symm_helper_y])

In [ ]:
P2A_phi = Polygon([symm_helper_l00[1], phi(P1b), P1a, u1])
P2B_phi = Polygon([symm_helper_l00[0], u4, P1b])
P2_phi = MultiPolygon([P2A_phi, P2B_phi])

In [ ]:
P3A_phi = Polygon([phi(P3a), P1a, symm_helper_l00[0], u3])
P3B_phi = Polygon([u2, phi(P1b), symm_helper_l00[1]])
P3_phi = MultiPolygon([P3A_phi, P3B_phi])

In [ ]:
MultiPolygon([P1A_phi, P1B_phi, P2A_phi, P2B_phi, P3A_phi, P3B_phi])

In [ ]:
partition_phi = [P1_phi, P2_phi, P3_phi]

In [ ]:
def phi_non_identified(x: np.array) -> np.array:
    return np.dot(A, x)

In [ ]:
def d_phi(x: np.array) -> np.array:
    return np.transpose(A)

In [ ]:
m_id = 1
gamma = 0.8
c = 3000
tau = 0.0001
target_state = np.array([0.5, 0.5])
dynamic_system = DynamicSystem(phi_non_identified, d_phi, m_id)
markov_decision_process = MarkovDecisionProcess(dynamic_system, partition, partition_vertices=partition_vertices, target_state=target_state, gamma=gamma)

In [ ]:
# add markov partition to overall experiment dictionary
baseline_partitions["markov_1"] = {}
baseline_partitions["markov_1"]["partition"] = partition
baseline_partitions["markov_1"]["partition_vertices"] = partition_vertices

In [ ]:
print("Estimate transition probabilities")
start_time_estimation = time.time()
markov_decision_process.estimate_probability_matrix_pi_method(c=c, tau=tau, max_sample_trials=1000)
total_time_estimation = time.time() - start_time_estimation
print(f"Transition probability estimation took {round(total_time_estimation, 2)}s")

print("Evaluate policy")
epsilon = 10e-6 # 2 * np.finfo(float).eps
start_time_evaluation = time.time()
V, num_iters, convergence_info = markov_decision_process.policy_evaluation(markov_decision_process.g, epsilon=epsilon)
total_time_evaluation = time.time() - start_time_evaluation
print(f"Policy evaluation took {total_time_evaluation}s")
print(f"Value function: {V}")
print(f"Number of iterations until convergence: {num_iters}")

In [ ]:
x_values = list(range(1, num_iters))
plt.plot(x_values, convergence_info["max_dist"][1:])
plt.xlabel("Iteration")
plt.ylabel("MAX-DIST")
plt.title("Maximal convergence distance over iterations")
plt.show()

In [ ]:
x_values = list(range(1, num_iters))
plt.plot(x_values, convergence_info["avg_dist"][1:])
plt.xlabel("Iteration")
plt.ylabel("L2-DIST")
plt.title("L2 convergence distance over iterations")
plt.show()

## Test policy evaluation for a baseline partition

In [ ]:
test_baseline_partition = baseline_partitions["horizontal"]["partition"]
test_baseline_partition_vertices = baseline_partitions["horizontal"]["partition_vertices"]
markov_decision_process = MarkovDecisionProcess(dynamic_system, test_baseline_partition, partition_vertices=test_baseline_partition_vertices, target_state=target_state, gamma=gamma)

In [ ]:
print("Estimate transition probabilities")
start_time_estimation = time.time()
markov_decision_process.estimate_probability_matrix_pi_method(c=c, tau=tau, max_sample_trials=1000)
total_time_estimation = time.time() - start_time_estimation
print(f"Transition probability estimation took {round(total_time_estimation, 2)}s")

print("Evaluate policy")
epsilon = 10e-6 # 2 * np.finfo(float).eps
start_time_evaluation = time.time()
V, num_iters, convergence_info = markov_decision_process.policy_evaluation(markov_decision_process.g, epsilon=epsilon)
total_time_evaluation = time.time() - start_time_evaluation
print(f"Policy evaluation took {total_time_evaluation}s")
print(f"Value function: {V}")
print(f"Number of iterations until convergence: {num_iters}")

In [ ]:
x_values = list(range(1, num_iters))
plt.plot(x_values, convergence_info["max_dist"][1:])
plt.xlabel("Iteration")
plt.ylabel("MAX-DIST")
plt.title("Maximal convergence distance over iterations")
plt.show()

In [ ]:
x_values = list(range(1, num_iters))
plt.plot(x_values, convergence_info["avg_dist"][1:])
plt.xlabel("Iteration")
plt.ylabel("L2-DIST")
plt.title("L2 convergence distance over iterations")
plt.show()

## Experiment with all partitions

In [ ]:
gamma = 0.8

for experiment_run in baseline_partitions.keys():
    print("\n---------------------------------------\n")
    print(f"Experiment run with {experiment_run} partition")

    partition = baseline_partitions[experiment_run]["partition"]
    partition_vertices = baseline_partitions[experiment_run]["partition_vertices"]
    markov_decision_process = MarkovDecisionProcess(dynamic_system, partition, partition_vertices=partition_vertices, target_state=target_state, gamma=gamma)

    print("Estimate transition probabilities")
    start_time_estimation = time.time()
    markov_decision_process.estimate_probability_matrix_pi_method(c=c, tau=tau, max_sample_trials=1000)
    total_time_estimation = time.time() - start_time_estimation
    print(f"Transition probability estimation took {round(total_time_estimation, 2)}s")

    print("Evaluate policy")
    epsilon = 10e-6 # 2 * np.finfo(float).eps
    start_time_evaluation = time.time()
    V, num_iters, convergence_info = markov_decision_process.policy_evaluation(markov_decision_process.g, epsilon=epsilon)
    total_time_evaluation = time.time() - start_time_evaluation
    print(f"Policy evaluation took {total_time_evaluation}s")
    print(f"Value function: {V}")
    print(f"Number of iterations until convergence: {num_iters}")

    x_values = list(range(1, num_iters))
    plt.plot(x_values, convergence_info["max_dist"][1:])
    plt.xlabel("Iteration")
    plt.ylabel("MAX-DIST")
    plt.title("Maximal convergence distance over iterations")
    plt.show()

    x_values = list(range(1, num_iters))
    plt.plot(x_values, convergence_info["avg_dist"][1:])
    plt.xlabel("Iteration")
    plt.ylabel("L2-DIST")
    plt.title("L2 convergence distance over iterations")
    plt.show()